In [ ]:
import requests
import csv
import bs4
from bs4 import BeautifulSoup
from openpyxl import Workbook
from requests.packages.urllib3.exceptions import InsecureRequestWarning

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

#def PTTpagenum(page):          #https://www.ptt.cc/bbs/HatePolitics/index(page).html
#    start = page.find('index') #page的起始點
#    end = page.find('.html')   #page的終末點
#    pagenum = page[start + 5 : end] #起始點開始往後找五頁
#    return int(pagenum) + 1

#每一篇文章內容
def article_web_data(articleurl_url_list):
    #將資料存成csv       
    csv_file = Workbook()
    csv_active = csv_file.active
    #excel檔
    csv_active['A1'] = '作者'
    csv_active['B1'] = '標題'
    csv_active['C1'] = '文章發佈時間'
    csv_active['D1'] = '文章內容'
    csv_active['E1'] = '推文數'
    csv_active['F1'] = '噓文數'
    
    for article_info in articleurl_url_list:
        #print(articleweb) #article_url
        
        #cookie重複驗證 年齡限制 頁面
        jar = requests.cookies.RequestsCookieJar()        
        jar.set("over18", "1", domain="www.ptt.cc")        
        response = requests.get(article_info, cookies=jar).text    
        
        #文章
        article_html = BeautifulSoup(response)        
        article_detail = article_html.find("div", id="main-content")
        
        #判斷是否為Nonetype
        if isinstance(article_detail,bs4.element.Tag):
            values = article_detail.find_all("span", class_="article-meta-value")
            #print(values)
            
            #index out of range錯誤排除
            try:
                if len(values) == 0 or "公告" in values[2].text or "版規" in values[2].text or "新聞" in values[2].text:
                    continue
                else:
                    #print("作者:", values[0].text)
                    #print("看板:", values[1].text)
                    #print("文章標題:", values[2].text)
                    #print("發文時間:", values[3].text)

                    #文章內文處理

                    #扣除 作者、標題、時間
                    article_metaline = article_detail.find_all('div', class_= 'article-metaline')
                    #print(article_metaline)
                    for am in article_metaline:
                        #print(am)
                        am.extract()

                    #扣除 看板名稱
                    article_metaline_right = article_detail.find_all('div', class_= 'article-metaline-right')
                    for amr in article_metaline_right:
                        #print(amr)
                        amr.extract()

                    #扣除 文章尾部的發信站、文章網址
                    article_end = article_detail.find_all('span',class_= 'f2')
                    for ae in article_end:
                        ae.extract()

                    #扣除 留言
                    pushes = article_detail.find_all('div', class_= 'push')
                    #print(pushes)
                    goodpoint = 0
                    badpoint = 0
                    #supportpoint = 0

                    for push in pushes:
                        #print(push) #每一篇留言
                        if isinstance(push ,bs4.element.Tag):
                            pushtag = push.find("span", class_="push-tag").text
                            #print(pushtag)
                            if "推" in pushtag:
                                goodpoint += 1
                            elif "噓" in pushtag:
                                badpoint += 1
                                    
                            #移除留言
                            push.extract()
                    #print("推文數:", goodpoint)
                    #print("噓文數:", badpoint)
                    #print('--------------')
                    #print("補充數:", supportpoint)
                    #print(article_detail.text)
                    
                    csv_active.append([values[0].text, values[2].text, values[3].text, article_detail.text.replace('\n', ' '), goodpoint, badpoint])
                    
        
            except:
                continue                
    csv_file.save('xxx_RawData_final.xls')